In [ ]:
import config
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from IPython.display import display
import json
import datetime
import math

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_fscore_support, f1_score, cohen_kappa_score, average_precision_score, precision_recall_curve

from utils.clustering_eval_utils import get_metrics, get_metrics_pr, evaluate_all_as, evaluate_all_as_pr, ensemble_results, ensemble_results_pr

In [ ]:
STAGE = 'ss'
MOD_PREFIX = "mod_smallimg3"
NEPOCH = 'latest'


DATAPATH = config.OUTPUT_PATH
base_dir = config.RAW_DATA_PATH
img_path = config.SCHULTHESS_DATAPATH
proc_dir = config.PROC_DATA_PATH

# #for rawq:
feature = 'rawq'
# folder = "2025-11-19_hdbscan"
# run = "run10"  
# mapping_list = [0, 4, 2, 3, -1, 1]

methods = 'comb_modalities'
folder = '2026-01-17_hdbscan'
run = 'run28'

# feature = 'img_features'
# #for img features:
# folder = "2025-09-12_hdbscan"
# folder_date = folder.split('_')[0]
# run = "run92"
# mapping_list = [np.int64(0), np.int64(-1), np.int64(2), np.int64(1)]

# feature = 'img_raw'
# folder = "2025-09-13_hdbscan_img"
# run = "run32"
# mapping_list = [np.int64(0), np.int64(1), np.int64(2), np.int64(-1), np.int64(3), np.int64(4)]

# feature = 'agg'
# folder = "2025-08-11_hdbscan"
# run = 'run150'
# mapping_list = [np.int64(7),
#  np.int64(0),
#  np.int64(2),
#  np.int64(10),
#  np.int64(1),
#  np.int64(8),
#  np.int64(9),
#  np.int64(5),
#  np.int64(-1),
#  np.int64(6),
#  np.int64(4),
#  np.int64(3)]

anomalyscore_metric = "centre_mean"
cluster_col = "cluster_label"

In [ ]:
today = datetime.date.today()
folder_date = folder.split('_')[0]

filepath = os.path.join(proc_dir, folder, methods, run)
save_path = os.path.join(filepath, "img")
os.makedirs(save_path, exist_ok=True)

# try:
#     hdbscan_df = pd.read_csv(os.path.join(filepath, f'pipeline_{run}_umap_hdbscan_scaled_allpoints_wKL.csv'))
# except:
hdbscan_df = pd.read_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_severity_scores.csv'))
smote = pd.read_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_smote_clusters.csv'))

with open(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_model_info.json')) as f:
    model_info= json.load(f)

# Comparison KL-Score (AUC/SRC)

## Compare Cluster Label to KL-Score

In [ ]:
# order = [0, 4, 2, 1]
# order_wnoise = [0,4,2,-1,1]

In [ ]:
# # hdbscan_df_dropna = hdbscan_df.dropna(subset=['KL-Score', 'KL-Score2'])
# hdbscan_df_dropna = hdbscan_df.copy()
# hdbscan_df_dropna = hdbscan_df_dropna[hdbscan_df_dropna['KL-Score'] != -1]
# hdbscan_df_dropna = hdbscan_df_dropna[hdbscan_df_dropna['KL-Score2'] != -1]

# hdbscan_df_dropna['cluster_label2'] = hdbscan_df_dropna['cluster_label'].map(mapping_dict)
metrics_clkl = get_metrics(hdbscan_df[hdbscan_df['cluster_label']!=-1], 'cluster_label', label_name = 'KL-Score')
metrics_clkl2 = get_metrics(hdbscan_df[hdbscan_df['cluster_label']!=-1], 'old_cluster', label_name = 'KL-Score')

In [ ]:
print(metrics_clkl)

In [ ]:
print(metrics_clkl2)

In [ ]:
hdbscan_df_wonoise = hdbscan_df[hdbscan_df['cluster_label']!=-1]

In [ ]:
metrics_clkl_train = get_metrics(hdbscan_df_wonoise[hdbscan_df_wonoise['train_test']=='train'], 'cluster_label', label_name = 'KL-Score')
metrics_clkl_test = get_metrics(hdbscan_df_wonoise[hdbscan_df_wonoise['train_test']=='test'], 'cluster_label', label_name = 'KL-Score')

In [ ]:
metrics_clkl_train

In [ ]:
metrics_clkl_test

In [ ]:
metrics_clkl_train = get_metrics(hdbscan_df_wonoise[hdbscan_df_wonoise['train_test']=='train'], 'old_cluster', label_name = 'KL-Score')
metrics_clkl_test = get_metrics(hdbscan_df_wonoise[hdbscan_df_wonoise['train_test']=='test'], 'old_cluster', label_name = 'KL-Score')

In [ ]:
metrics_clkl_train

In [ ]:
metrics_clkl_test

In [ ]:
metrics_clkl_pr = get_metrics_pr(hdbscan_df_wonoise, 'cluster_label', label_name = 'KL-Score')
print(metrics_clkl_pr)

In [ ]:
metrics_clkl_pr = get_metrics_pr(hdbscan_df_wonoise, 'old_cluster', label_name = 'KL-Score')
print(metrics_clkl_pr)

'Osteophytes', 'Joint-Space-Narrowing', 'pain'

## Compare KL-Score to AS

In [ ]:
#get columns that end with _all
as_col = hdbscan_df.columns[hdbscan_df.columns.str.endswith('_all')]


# print(as_col)

In [ ]:
# as_col = df.iloc[:, 1:-2].columns

# use get_metrics to get src, auc, auc_mid, auc_mid2, auc_sev for all as_col and then calculate mean and std of those values
results_df = evaluate_all_as(hdbscan_df.dropna(subset=as_col), as_col, label_name='KL-Score')
# results_df2 = evaluate_all_as(hdbscan_df_dropna.dropna(subset=as_col), as_col, label_name='KL-Score2')

In [ ]:
ensemble_results(results_df)

In [ ]:
results_df2 = evaluate_all_as_pr(hdbscan_df.dropna(subset=as_col), as_col, label_name='KL-Score')
ensemble_results_pr(results_df2)

## Compare Cluster Label to AS

In [ ]:
# hdbscan_df = hdbscan_df.fillna(-1)

In [ ]:
results_df3 = evaluate_all_as(hdbscan_df[hdbscan_df['cluster_label']!=-1].dropna(subset=['mean']), as_col, label_name='cluster_label')

In [ ]:
results_df4 = evaluate_all_as(hdbscan_df[hdbscan_df['old_cluster']!=-1].dropna(subset=['mean']), as_col, label_name='old_cluster')

In [ ]:
ensemble_results(results_df3)

In [ ]:
ensemble_results(results_df4)

## Boxplot

In [ ]:
dfs = hdbscan_df.dropna(subset=['mean'])

In [ ]:
hdbscan_df['cluster_label'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))

sns.set_theme(style="whitegrid", font_scale=1.2)
sns.boxplot(
    data=dfs,
    x='cluster_label',
    y='mean',
    palette='Set3',    # nice pastel palette
    width=0.6
)

plt.xlabel("Cluster Label", fontsize=14)
plt.ylabel("Mean Anomaly Score", fontsize=14)
plt.title("Anomaly Score Distribution per Cluster", fontsize=16)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))

sns.set_theme(style="whitegrid", font_scale=1.2)
sns.boxplot(
    data=dfs,
    x='old_cluster',
    y='mean',
    palette='Set3',    # nice pastel palette
    width=0.6
)

plt.xlabel("Cluster Label", fontsize=14)
plt.ylabel("Mean Anomaly Score", fontsize=14)
plt.title("Anomaly Score Distribution per Cluster", fontsize=16)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.show()

## Compare KL-Score to MRI

In [ ]:
mri = pd.read_csv(os.path.join(base_dir, '2025-09-25_mrismall.csv'))

hdbscan_df = hdbscan_df.merge(mri, on='id', how='left')

In [ ]:
results_kl_mri = evaluate_all_as(hdbscan_df, ['KL-Score'], label_name='mri_cart_yn')
print('Cartilage:')
ensemble_results(results_kl_mri)
print()
results_kl_mri = evaluate_all_as(hdbscan_df, ['KL-Score'], label_name='mri_osteo_yn')
print('Osteophytes:')
ensemble_results(results_kl_mri)
print()
results_kl_mri = evaluate_all_as(hdbscan_df, ['KL-Score'], label_name='mri_bml_yn')
print('Bone Marrow Lesions:')
ensemble_results(results_kl_mri)

In [ ]:
results_df2_pr = evaluate_all_as_pr(hdbscan_df.dropna(subset=as_col), ['KL-Score'], label_name='mri_cart_yn')
print('MRI Cartilage:')
ensemble_results_pr(results_df2_pr)
results_df2_pr = evaluate_all_as_pr(hdbscan_df.dropna(subset=as_col), ['KL-Score'], label_name='mri_osteo_yn')
print('MRI Osteophytes:')
ensemble_results_pr(results_df2_pr)
results_df2_pr = evaluate_all_as_pr(hdbscan_df.dropna(subset=as_col), ['KL-Score'], label_name='mri_bml_yn')
print('MRI BML:')
ensemble_results_pr(results_df2_pr)

## Compare Cluster Label to MRI

In [ ]:
results_kl_mri = evaluate_all_as(hdbscan_df, ['cluster_label'], label_name='mri_cart_yn')
print('Cartilage:')
ensemble_results(results_kl_mri)
print()
results_kl_mri = evaluate_all_as(hdbscan_df, ['cluster_label'], label_name='mri_osteo_yn')
print('Osteophytes:')
ensemble_results(results_kl_mri)
print()
results_kl_mri = evaluate_all_as(hdbscan_df, ['cluster_label'], label_name='mri_bml_yn')
print('Bone Marrow Lesions:')
ensemble_results(results_kl_mri)

In [ ]:
results_kl_mri = evaluate_all_as_pr(hdbscan_df, ['cluster_label'], label_name='mri_cart_yn')
print('Cartilage:')
ensemble_results_pr(results_kl_mri)
print()
results_kl_mri = evaluate_all_as_pr(hdbscan_df, ['cluster_label'], label_name='mri_osteo_yn')
print('Osteophytes:')
ensemble_results_pr(results_kl_mri)
print()
results_kl_mri = evaluate_all_as_pr(hdbscan_df, ['cluster_label'], label_name='mri_bml_yn')
print('Bone Marrow Lesions:')
ensemble_results_pr(results_kl_mri)

In [ ]:
results_kl_mri = evaluate_all_as(hdbscan_df, ['old_cluster'], label_name='mri_cart_yn')
print('Cartilage:')
ensemble_results(results_kl_mri)
print()
results_kl_mri = evaluate_all_as(hdbscan_df, ['old_cluster'], label_name='mri_osteo_yn')
print('Osteophytes:')
ensemble_results(results_kl_mri)
print()
results_kl_mri = evaluate_all_as(hdbscan_df, ['old_cluster'], label_name='mri_bml_yn')
print('Bone Marrow Lesions:')
ensemble_results(results_kl_mri)

In [ ]:
results_kl_mri = evaluate_all_as_pr(hdbscan_df, ['old_cluster'], label_name='mri_cart_yn')
print('Cartilage:')
ensemble_results_pr(results_kl_mri)
print()
results_kl_mri = evaluate_all_as_pr(hdbscan_df, ['old_cluster'], label_name='mri_osteo_yn')
print('Osteophytes:')
ensemble_results_pr(results_kl_mri)
print()
results_kl_mri = evaluate_all_as_pr(hdbscan_df, ['old_cluster'], label_name='mri_bml_yn')
print('Bone Marrow Lesions:')
ensemble_results_pr(results_kl_mri)

## Aggregating Cluster Values

In [ ]:
mapping_list

In [ ]:
#could try mapping it more